# People, States, and Arrays
Starsim is a framework for creating agent-based models, and the `People` class is where we store the agents, so it should come as no surprise that this class serves as the fundamental heart of any Starsim model. In this page we provide alternative pathways for creating people and some guidance on how to adapt these workflows depending on your needs. 

We start by giving an overview on this page of Starsim's custom `Arr` (array) classes, which are a separate but related Starsim class designed to neatly track data about people. 

## Starsim States and Arrays
Starsim has a set of custom array classes for recording information about each agent in the population. The two fundamental types of array for storing such infomation are the `State` class, which is a Boolean array, and the `FloatArr` class, which stores numbers (we don't distinguish between floats and integers, so all numbers are stored in these arrays). Each of these is a subclass of the Starsim `Arr` class.

The `Arr` class in Starsim is optimized for three key tasks that are common to almost all Starsim models: 
1. dynamic growth: as the population grows over time, the size of the arrays dynamically update in a way that avoids costly concatenation operations;
2. indexing: over time, there are agents in the population who die. It is desirable for these agents to remain in the arrays so that we can continue to access data about them, but the indexing is set up so that dead agents are automatically excluded from most operations.
3. stochastic states: we often want to set the values of a state by sampling from a random variable (e.g. sex might be drawn as a Bernoulli random variable). Starsim's `Arr` class can be initialized with a random variables; we will provide examples of this below.

## Creating default people
When you create a sim, it automatically creates `People`, and you can use the `n_agents` argument to control the population size:

In [ ]:
import numpy as np
import pandas as pd
import starsim as ss 
sim = ss.Sim(n_agents=1000)  # Create a sim with default people
sim.init()

The `People` that are added to the `Sim` come with the following default states and arrays:
* 'alive', a `State` that records whether each agent is alive
* 'female', a `State` that records whether each agent is female
* 'age', a `FloatArr` that records agent ages
* `ti_dead`, a `FloatArr` that records the time of death, NaN by default
* `scale`, a `FloatArr` that records the number of people that each agent represents; 1 by default.

## Creating custom people
Rather than relying on the `Sim` to create people, you can create your own `People` and add them to the `Sim` as a separate argument. The example below is equivalent to the one immediately above:

In [ ]:
people = ss.People(1000)
sim = ss.Sim(people=people)

The main reason to create custom people is if you want to specify a particular age/sex distribution. The following example creates a population with the age distribution of Nigeria:

In [ ]:
age_data = pd.read_csv('test_data/nigeria_age.csv')
ppl = ss.People(n_agents=10e3, age_data=age_data)  

Another reason to create custom people is if there are additional attributes that you want to track. Let's say we want to add a state to track urban/rural status. The example below also illustrates how you can add a stochastic state whose values are sampled from a distribution.

In [ ]:
def urban_function(n):
    """ Make a function to randomly assign people to urban/rural locations """ 
    return np.random.choice(a=[True, False], p=[0.5, 0.5], size=n)

urban = ss.State('urban', default=urban_function)
ppl = ss.People(10, extra_states=urban)  # Create 10 people with this state
sim = ss.Sim(people=ppl)
sim.init()  # Initialize the sim --> essential step to create the people and sample states
print(f'Number of urban people: {np.count_nonzero(sim.people.urban)}')

## Modifying people with modules
We saw an example above of adding a custom state to people. However, a far more common way to add states to people is by adding a module to the `Sim`. All the states of the modules will automatically get added to the main `People` instance. 

In [ ]:
ppl = ss.People(10)
ppl.add_module(ss.HIV())
sim = ss.Sim(people=ppl)
sim.init()
print(f'Number of HIV-infected people: {np.count_nonzero(sim.people.hiv.infected)}')

When states or arrays are added by modules, they are stored as dictionaries under the name of that module.

Note that the Starsim `Arr` class can be used like a Numpy array, with all the standard arithmetic operations like sums, mean, counting, etc.